In [1]:
#imports
import argparse
import torch
import pickle
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Training settings
cuda = torch.cuda.is_available()

seed = 10


kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
n_classes = 10
z_dim = 2
X_dim = 784
y_dim = 10
train_batch_size = 100
valid_batch_size = 100
N = 1000
epochs = 10 #500

In [3]:
##################################
# Load data and create Data loaders
##################################
def load_data(data_path='../data/'):
    print('loading data!')
    trainset_unlabeled = pickle.load(open(data_path + "train_unlabeled.p", "rb"))
    # Set -1 as labels for unlabeled data
    trainset_unlabeled.train_labels = torch.from_numpy(np.array([-1] * 47000))

    train_unlabeled_loader = torch.utils.data.DataLoader(trainset_unlabeled,
                                                         batch_size=train_batch_size,
                                                         shuffle=True, **kwargs)

    return train_unlabeled_loader

In [4]:

##################################
# Define Networks
##################################
# Encoder
class Q_net(nn.Module):
    def __init__(self):
        super(Q_net, self).__init__()
        self.lin1 = nn.Linear(X_dim, N)
        self.lin2 = nn.Linear(N, N)
        # Gaussian code (z)
        self.lin3gauss = nn.Linear(N, z_dim)

    def forward(self, x):
        x = F.dropout(self.lin1(x), p=0.2, training=self.training)
        x = F.relu(x)
        x = F.dropout(self.lin2(x), p=0.2, training=self.training)
        x = F.relu(x)
        xgauss = self.lin3gauss(x)

        return xgauss


In [5]:

# Decoder
class P_net(nn.Module):
    def __init__(self):
        super(P_net, self).__init__()
        self.lin1 = nn.Linear(z_dim, N)
        self.lin2 = nn.Linear(N, N)
        self.lin3 = nn.Linear(N, X_dim)

    def forward(self, x):
        x = self.lin1(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin3(x)
        return F.sigmoid(x)

In [6]:

class D_net_gauss(nn.Module):
    def __init__(self):
        super(D_net_gauss, self).__init__()
        self.lin1 = nn.Linear(z_dim, N)
        self.lin2 = nn.Linear(N, N)
        self.lin3 = nn.Linear(N, 1)

    def forward(self, x):
        x = F.dropout(self.lin1(x), p=0.2, training=self.training)
        x = F.relu(x)
        x = F.dropout(self.lin2(x), p=0.2, training=self.training)
        x = F.relu(x)

        return F.sigmoid(self.lin3(x))

In [7]:

####################
# Utility functions
####################
def save_model(model, filename):
    print('Best model so far, saving it...')
    torch.save(model.state_dict(), filename)


def report_loss(epoch, D_loss_gauss, G_loss, recon_loss):
    '''
    Print loss
    '''
    print('Epoch-{}; D_loss_gauss: {:.4}; G_loss: {:.4}; recon_loss: {:.4}'.format(epoch,
                                                                                   D_loss_gauss.data[0],
                                                                                   G_loss.data[0],
                                                                                   recon_loss.data[0]))


def create_latent(Q, loader):
    '''
    Creates the latent representation for the samples in loader
    return:
        z_values: numpy array with the latent representations
        labels: the labels corresponding to the latent representations
    '''
    Q.eval()
    labels = []

    for batch_idx, (X, target) in enumerate(loader):

        X = X * 0.3081 + 0.1307
        # X.resize_(loader.batch_size, X_dim)
        X, target = Variable(X), Variable(target)
        labels.extend(target.data.tolist())
        if cuda:
            X, target = X.cuda(), target.cuda()
        # Reconstruction phase
        z_sample = Q(X)
        if batch_idx > 0:
            z_values = np.concatenate((z_values, np.array(z_sample.data.tolist())))
        else:
            z_values = np.array(z_sample.data.tolist())
    labels = np.array(labels)

    return z_values, labels


In [8]:

       
####################
# Train procedure
####################
def train(P, Q, D_gauss, P_decoder, Q_encoder, Q_generator, D_gauss_solver, data_loader):
    '''
    Train procedure for one epoch.
    '''
    TINY = 1e-15
    # Set the networks in train mode (apply dropout when needed)
    Q.train()
    P.train()
    D_gauss.train()

    # Loop through the labeled and unlabeled dataset getting one batch of samples from each
    # The batch size has to be a divisor of the size of the dataset or it will return
    # invalid samples
    for X, target in data_loader:
        # Load batch and normalize samples to be between 0 and 1
        X = X * 0.3081 + 0.1307
        X.resize_(train_batch_size, X_dim)
        
        X, target = Variable(X), Variable(target)
        if cuda:
            X, target = X.cuda(), target.cuda()

        # Init gradients
        P.zero_grad()
        Q.zero_grad()
        D_gauss.zero_grad()

        #######################
        # Reconstruction phase
        #######################
        z_sample = Q(X)
        X_sample = P(z_sample)
        recon_loss = F.binary_cross_entropy(X_sample + TINY, X.resize(train_batch_size, X_dim) + TINY)

        recon_loss.backward()
        P_decoder.step()
        Q_encoder.step()

        P.zero_grad()
        Q.zero_grad()
        D_gauss.zero_grad()

        #######################
        # Regularization phase
        #######################
        # Discriminator
        Q.eval()
        z_real_gauss = Variable(torch.randn(train_batch_size, z_dim) * 5.)
        if cuda:
            z_real_gauss = z_real_gauss.cuda()

        z_fake_gauss = Q(X)

        D_real_gauss = D_gauss(z_real_gauss)
        D_fake_gauss = D_gauss(z_fake_gauss)
        D_loss = -torch.mean(torch.log(D_real_gauss + TINY) + torch.log(1 - D_fake_gauss + TINY))
        D_loss.backward()
        D_gauss_solver.step()

        P.zero_grad()
        Q.zero_grad()
        D_gauss.zero_grad()

        # Generator
        Q.train()
        z_fake_gauss = Q(X)

        D_fake_gauss = D_gauss(z_fake_gauss)
        G_loss = -torch.mean(torch.log(D_fake_gauss + TINY))

        G_loss.backward()
        Q_generator.step()

        P.zero_grad()
        Q.zero_grad()
        D_gauss.zero_grad()
    return D_loss, G_loss, recon_loss



In [9]:
def generate_model( train_unlabeled_loader):
    torch.manual_seed(10)

    if cuda:
        Q = Q_net().cuda()
        P = P_net().cuda()
        D_gauss = D_net_gauss().cuda()
    else:
        Q = Q_net()
        P = P_net()
        D_gauss = D_net_gauss()

    # Set learning rates
    gen_lr = 0.0001
    reg_lr = 0.00005

    # Set optimizators
    P_decoder = optim.Adam(P.parameters(), lr=gen_lr)
    Q_encoder = optim.Adam(Q.parameters(), lr=gen_lr)

    Q_generator = optim.Adam(Q.parameters(), lr=reg_lr)
    D_gauss_solver = optim.Adam(D_gauss.parameters(), lr=reg_lr)

    for epoch in range(epochs):
        D_loss_gauss, G_loss, recon_loss = train(P, Q, D_gauss, P_decoder, Q_encoder,
                                                 Q_generator,
                                                 D_gauss_solver,
                                                 train_unlabeled_loader)
        if epoch % 1 == 0:#chaged here!
            report_loss(epoch, D_loss_gauss, G_loss, recon_loss)

    return Q, P

In [10]:
if __name__ == '__main__':
    train_unlabeled_loader = load_data()
    Q, P = generate_model( train_unlabeled_loader)

loading data!
3000
750
Epoch-0; D_loss_gauss: 1.13; G_loss: 1.284; recon_loss: 0.2118
Epoch-1; D_loss_gauss: 1.203; G_loss: 0.9137; recon_loss: 0.2222
Epoch-2; D_loss_gauss: 1.236; G_loss: 0.9061; recon_loss: 0.2116
Epoch-3; D_loss_gauss: 1.265; G_loss: 0.7916; recon_loss: 0.2105
Epoch-4; D_loss_gauss: 1.254; G_loss: 0.8106; recon_loss: 0.2087


Process Process-6:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/minato/.virtualenvs/dlcv/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [13]:
import PIL.Image
from io import BytesIO
import IPython.display
import numpy as np
import math

def hardcodedTransform(array):
    return (array * 0.3081 + 0.1307)

def transformForNetwork(array):
    return Variable(torch.FloatTensor( hardcodedTransform(array.reshape(28*28)))).cuda()

def showarray(a, fmt='png'):
    a = np.uint8(hardcodedTransform(a)*256)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [15]:
from PIL import Image
from IPython.display import Image, display

TINY = 1e-15

data_path='../data/'
trainset_unlabeled = pickle.load(open(data_path + "train_unlabeled.p", "rb"))
trainset_unlabeled.train_labels = torch.from_numpy(np.array([-1] * 47000))
my_loader = torch.utils.data.DataLoader(trainset_unlabeled,
                                                     batch_size=1,
                                                     shuffle=True, **kwargs)
i = 0
max_images = 10
out = []
imgList = np.zeros((10,28,28))
reconimgList = np.zeros((10,28,28))
imgZValue = np.zeros((10,2))
for X, target in my_loader:
    if i>=max_images:
        break
    out = X.numpy().reshape(28,28)
    imgList[i] = out
    t = Q(transformForNetwork(out))
    imgZValue[i] = t.data.cpu().numpy()
    reconimg =  P(t)
    reconimgList[i] = reconimg.cpu().data.numpy().reshape(28,28)
    showarray(out)
    showarray(reconimgList[i]*255)
    print(imgZValue[i])
    i+=1

[-7.2910862  -5.52014303]


[4.59525967 9.13639927]


[0.23103715 8.86695957]


[1.46240759 2.49664259]


[0.81836706 2.89056945]


[ 0.08262368 -2.98100066]


[-3.94690704  1.25019276]


[-11.85185432   2.34102178]


[0.33046523 2.65030289]


[-3.52706218 -0.11406142]


In [13]:
showarray(imgList[0])

In [14]:
diff_matrix = np.zeros((max_images,max_images))
for i in range(0,max_images):
    for j in range(0,max_images):
        #print(diff_matrix[i,j])
        diff_matrix[i,j]=math.sqrt(sum(np.power(imgZValue[i] - imgZValue[j],2)))
        #print(imgZValue[i] - imgZValue[j])
print(diff_matrix)

[[  0.           5.41346046  15.27529478   5.12961441   7.40172941
   12.75295141   9.85286453   5.23819878  12.18814157   6.53840715]
 [  5.41346046   0.          11.31014073   2.14682875   2.84853302
    9.52751523   9.76405124   6.04871786   6.7758124    2.45547021]
 [ 15.27529478  11.31014073   0.          13.35476739   8.47162997
    3.1041629   20.64185988  17.35519257   8.67820083   9.08482203]
 [  5.12961441   2.14682875  13.35476739   0.           4.95246182
   11.66691387   7.62384096   4.04254185   7.79648572   4.6008404 ]
 [  7.40172941   2.84853302   8.47162997   4.95246182   0.           6.73924698
   12.48988791   8.8964105    5.60563152   0.91901629]
 [ 12.75295141   9.52751523   3.1041629   11.66691387   6.73924698   0.
   19.22871574  15.50060606   8.95609448   7.10553395]
 [  9.85286453   9.76405124  20.64185988   7.62384096  12.48988791
   19.22871574   0.           4.64047579  13.3879637   12.2191704 ]
 [  5.23819878   6.04871786  17.35519257   4.04254185   8.89641